# Task 3: Helper notebook for loading the data and saving the predictions

In [ ]:
import pickle
import gzip
import numpy as np
import os

### Helper functions

In [ ]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [ ]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

### Load data, make predictions and save prediction in correct format

In [ ]:
# load data
train_data = load_zipped_pickle("train.pkl")
test_data = load_zipped_pickle("test.pkl")
samples = load_zipped_pickle("sample.pkl")


In [ ]:
from sklearn.preprocessing import RobustScaler
import elasticdeform

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np


## Preprocess Data

In [ ]:
# from torchvision.transforms import Resize, RandomHorizontalFlip, RandomVerticalFlip

import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import cv2


train_videos = []
train_labels = []
train_boxes = []
for data in train_data:
    
    train_videos.append(np.transpose(data['video'], (2,0,1))[data['frames']])
    train_boxes.append((data['box']))
    y = (np.transpose(data['label'],(2,0,1))[data['frames']])
    train_labels.append(y)

#
target_size = (128, 128)


def resize_video_and_mask(video, mask, target_size):
    # Resize the image
    # video = np.array(video)
    # print(video)
    resized_video = []
    resized_labels = []
    for image in video:
        resized_video.append(cv2.resize(image, target_size, interpolation=cv2.INTER_CUBIC))

    for l in mask:
        # resized_labels.append(cv2.resize(l, target_size))
        resized_labels.append((cv2.resize(l.astype(np.uint8), target_size, interpolation=cv2.INTER_CUBIC)).astype(bool))

    # Resize the mask using nearest-neighbor interpolation
    # mask = np.array(mask)
    # masks = []
    # for m in mask:
    # mask = (cv2.resize(mask.astype(np.uint8), target_size, interpolation=cv2.INTER_NEAREST)).astype(bool)

    # Convert the resized mask back to boolean values
    # resized_mask = resized_mask.astype(bool)

    return resized_video, resized_labels


# Define an elastic deformation transform
def elastic_deformation_transform(video, mask):
    # video = video.transpose(2, 0, 1)  # PyTorch expects channels-first format
    # deformation_field = elasticdeform.deformation_map(target_size, alpha=30.0, sigma=3)
    displacement = np.ones((2,128,128)) * np.random.uniform(-10,10)# + np.ones((2,192,192)) * 0.00001

    # box = elasticdeform.deform_grid(mask, displacement=displacement, mode='constant', order=1.5)
        # x.append()
    x = (elasticdeform.deform_grid(video, displacement=displacement, mode='constant', order=1.5))

    y = (elasticdeform.deform_grid(mask, displacement=displacement, mode='constant', order=1.5))       

        
    return x, y

def elastic_deformation_transform2(video, mask):
    # video = video.transpose(2, 0, 1)  # PyTorch expects channels-first format
    # deformation_field = elasticdeform.deformation_map(target_size, alpha=30.0, sigma=3)
    displacement = np.random.randn(2,128,128) # + np.ones((2,192,192)) * 0.00001

    # box = elasticdeform.deform_grid(mask, displacement=displacement, mode='constant', order=1.5)
        # x.append()
    x = (elasticdeform.deform_grid(video, displacement=displacement, mode='constant', order=0))

    y = (elasticdeform.deform_grid(mask, displacement=displacement, mode='constant', order=0))       

        
    return x, y

def rotation(video, mask):
    # video = video.transpose(2, 0, 1)  # PyTorch expects channels-first format
    # deformation_field = elasticdeform.deformation_map(target_size, alpha=30.0, sigma=3)
    displacement = np.zeros((2,128,128))# + np.ones((2,192,200)) * 0.00001
    # box = elasticdeform.deform_grid(mask, displacement=displacement, mode='constant', order=1.5)
        # x.append()
    x = (elasticdeform.deform_grid(video, displacement=displacement, mode='constant', order=1, rotate=np.random.uniform(-30,30)))
        # x.append()
    y = (elasticdeform.deform_grid(mask, displacement=displacement, mode='constant', order=1, rotate=np.random.uniform(-30,30)))
        
    return x, y

def zoom(video, mask):
    # video = video.transpose(2, 0, 1)  # PyTorch expects channels-first format
    # deformation_field = elasticdeform.deformation_map(target_size, alpha=30.0, sigma=3)
    flip_transform = transforms.RandomVerticalFlip()
    horizontal_transform = transforms.RandomHorizontalFlip()

    displacement = np.zeros((2,128,128))# + np.ones((2,192,200)) * 0.00001
    # box = elasticdeform.deform_grid(mask, displacement=displacement, mode='constant', order=1.5)
    x = (elasticdeform.deform_grid(video, displacement=displacement, mode='constant', order=1, zoom=np.random.uniform(0.8, 1.2)))
    y = (elasticdeform.deform_grid(mask, displacement=displacement, mode='constant', order=1, zoom=np.random.uniform(0.8,1.2)))
        
    return x, y

def stack_transforms(video,mask):
        deformed_video, deformed_mask = elastic_deformation_transform(video, mask)
        #add deformed
        rotatevid, rotatmask = rotation(deformed_video, deformed_mask)
        #add deformed

        zoomvid, zoomlab = zoom(rotatevid, rotatmask)
        return zoomvid,  zoomlab
    
def medianblur(frame):
    return cv2.medianBlur(frame,3)



resized_videos = []
resized_labels = []
for i in range(len(train_videos)):
    #resize
    resized_video, resized_mask = resize_video_and_mask(train_videos[i], train_labels[i], target_size)

    #add resized
    resized_videos.append(resized_video)
    resized_labels.append(resized_mask)


    
    for j in range(25):
        for idx in range(len(resized_video)):
            frame = resized_video[idx]
            label_frame = resized_mask[idx]

            newvideo = []
            newlabel = []
            
            stackvid, stacklab = stack_transforms(frame, label_frame)
            newvideo.append(stackvid)
            newlabel.append(stacklab)

            resized_videos.append(newvideo)
            resized_labels.append(newlabel)

    

## Visualize images

In [ ]:
from PIL import Image
import numpy as np
from IPython.display import display


# Assuming 'image_matrix' is your image matrix (NumPy array)
for i in range(1,len(resized_videos)):
    # image_video = resized_videos[i][20]
    # print(resized_labels[i][2])
    image_matrix = (resized_videos[i][4])
# Convert the NumPy array to a Pillow Image
    image = Image.fromarray(image_matrix)

# Save the image as a JPEG file
# image.save('output.jpg')
    display(image)

# Create DataLoader for training

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

X = []
y = []
for i in range(len(resized_videos)):
    for frame in resized_videos[i]:
        frame = cv2.normalize(frame,  np.zeros(frame.shape), 0, 255, cv2.NORM_MINMAX)
        X.append(np.expand_dims(frame,0))
    for frame in resized_labels[i]:
         y.append(np.expand_dims(frame,0))       
X = np.stack(X,0)
y = np.stack(y,0)

print(X.shape)
print(y.shape)

# print(np.max(X[0][0]))

# X_normalized = X / 255.0  # Assuming pixel values are in the range [0, 255]
# X_normalized = (X_normalized - 0.5) / 0.5
# # X_train_normalized = (X - mean_value) / std_value
    
# Assuming X_train and y_train are your training data and labels
# X_train and y_train should be PyTorch tensors

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Convert data to PyTorch tensors
X_train, y_train, X_val, y_val = map(torch.tensor, (X_train, y_train, X_val, y_val))

X_train = X_train.to(torch.float32)  # Convert to float32
y_train = y_train.to(torch.float32)  # Convert to float32
X_val = X_val.to(torch.float32)  # Convert to float32
y_val = y_val.to(torch.float32)  # Convert to float32
# Create DataLoader for training and validation data
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


## Define Loss

In [ ]:
import torch
import torch.nn.functional as F

def jaccard_similarity(y_true, y_pred):
    intersection = torch.sum(y_true * y_pred)
    union = torch.sum(y_true) + torch.sum(y_pred) - intersection
    return intersection / union

def jaccard_loss(y_true, y_pred):
    return -jaccard_similarity(y_true, y_pred)





## Define Model

In [ ]:

import torch.nn as nn
import torch.nn.functional as F
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Contracting path
        self.conv1 = self.conv_block(in_channels, 64)


        self.conv2 = self.conv_block(64, 128)
        self.conv3 = self.conv_block(128, 256)
        self.conv4 = self.conv_block(256, 512)

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        self.upconv4 = self.upconv_block(1024, 512)
        # Expansive path
        self.conv5 = self.conv_block(1024, 512)
        self.upconv3 = self.upconv_block(512, 256)
        self.conv6 = self.conv_block(512, 256)
        self.upconv2 = self.upconv_block(256, 128)
        self.conv7 = self.conv_block(256, 128)
        self.upconv1 = self.upconv_block(128, 64)
        self.conv8 = self.conv_block(128, 64)
        # Output layer
        self.outconv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=005)  # Adjust the dropout probability as needed
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2, padding=0),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            # nn.Dropout(p=0.05)  # Adjust the dropout probability as needed
        )

    def forward(self, x):
        # Contracting path
        conv1 = self.conv1(x)
        conv2 = self.conv2(F.max_pool2d(conv1, 2))
        conv3 = self.conv3(F.max_pool2d(conv2, 2))
        conv4 = self.conv4(F.max_pool2d(conv3, 2))

        # Bottleneck
        bottleneck = self.bottleneck(F.max_pool2d(conv4, 2))

        # Expansive path
        upconv4 = self.upconv4(bottleneck)
        upconv3 = self.upconv3(self.conv5(torch.cat([conv4, upconv4], 1)))
        upconv2 = self.upconv2(self.conv6(torch.cat([conv3, upconv3], 1)))
        upconv1 = self.upconv1(self.conv7(torch.cat([conv2, upconv2], 1)))
        # Output layer
        out = self.outconv(self.conv8(torch.cat([conv1,upconv1],1)))

        return F.sigmoid(out)



# Train Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')


model = UNet(in_channels=1, out_channels=1)
# model.load_state_dict(torch.load('model_checkpoint_epoch_1.pt'))
model.to(device)
# Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()  # Adjust the loss function based on your task
# criterion = nn.BCEWithLogitsLoss()
criterion = jaccard_loss

optimizer = optim.Adam(model.parameters(), lr=3e-4)

# Training loop
num_epochs = 200
best_val_loss = float('inf')


for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        # Unpack the batch
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss = criterion(targets, outputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        for inputs_val, targets_val in val_loader:
            inputs_val, targets_val = inputs_val.to(device), targets_val.to(device)
            val_outputs = model(inputs_val)
            val_loss = criterion(targets_val, val_outputs)

    avg_val_loss = val_loss / len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Val Loss: {val_loss.item()}')
    if val_loss.item() <= -0.4:
        torch.save(model.state_dict(), f"model_checkpoint_epoch_{epoch + 1}.pt")
   

## Train on Validation Set

In [ ]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = UNet(1,1)
print("2")

model.to(device)
model.load_state_dict(torch.load('val_model_checkpoint_epoch_24.pt'))
criterion = jaccard_loss

optimizer = optim.Adam(model.parameters(), lr=3e-4)
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    count = 0
    for inputs, targets in val_loader:
        # Unpack the batch
        inputs, targets = inputs.to(device), targets.to(device)

        # Iterate over each sample in the batch
        # for sample_input, sample_target in zip(inputs, targets):
            # Forward pass
        # input_image = inputs.unsqueeze(0).unsqueeze(0) # Add batch and channel dimensions
        outputs = model(inputs)

        # Compute loss
        # sample_target = sample_target.unsqueeze(0).unsqueeze(0)
        # print(targets)
        # print(targets.float())
        loss = criterion(targets, outputs)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if count % 10 == 0:
        #     print(loss.item())
        count += 1
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
    if loss.item() <= -0.5:
        torch.save(model.state_dict(), f"val_model_checkpoint_epoch_{epoch + 1}.pt")

## Predict on Test Data

In [52]:
import cv2
from torch.utils.data import DataLoader, TensorDataset

test_videos = []
for data in test_data:
   
    test_videos.append(np.transpose(data['video'], (2,0,1)))

test = []
for video in test_videos:
    for image in video:
        test.append(image)

target_size = (128,128)
resized_test = []
for image in test:
    resized_test.append(cv2.resize(image, target_size))
norm = []
for frame in resized_test:
    im = cv2.normalize(frame,  np.zeros(frame.shape), 0, 255, cv2.NORM_MINMAX)
    norm.append(np.expand_dims(im,0))

X_test = np.stack(norm,0)
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(X_test.shape)
model = UNet(1,1)
model.load_state_dict(torch.load('val_model_checkpoint_epoch_24.pt'))
model.eval()
model.to(device)
X_test = torch.from_numpy(X_test)
X_test = X_test.to(torch.float32)  # Convert to float32
X_test = X_test.to(device)

X_test_d = TensorDataset(X_test)
batch_size = 32
test_loader = DataLoader(X_test_d, batch_size=batch_size, shuffle=False)

predictions = []
for i in test_loader:
    pred = model(i[0])
    for frame in pred:
        predictions.append((frame[0] >= 0.6).cpu())





(1507, 1, 128, 128)


## Visualize predicitions

In [ ]:
from PIL import Image
import numpy as np
from IPython.display import display


# Assuming 'image_matrix' is your image matrix (NumPy array)
for p in (predictions):
    # image_video = resized_videos[i][20]
    # print(resized_labels[i][2])
    p = np.array(p)
    print(p.shape)
    image_matrix = p
# Convert the NumPy array to a Pillow Image
    image = Image.fromarray(image_matrix)

    display(image)

# Save the image as a JPEG file
# image.save('output.jpg')
    
    

## Post Processing with Closing

In [53]:
from sklearn.cluster import KMeans
import cv2


post_pred_test = []
for p in predictions:
  
  labels = np.reshape(p, (128,128))
  labels = np.uint8(labels)
    
  kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
  closing = cv2.morphologyEx(labels, cv2.MORPH_CLOSE, kernel)
 

  post_pred_test.append(closing >= 0.6)

In [54]:

final = []
for resized, orig in zip(post_pred_test, test):
      
      orig = np.array(orig)
      resized = np.array(resized)
      print(orig.shape)
      original_size = cv2.resize(resized.astype(np.uint8), (orig.shape[1], orig.shape[0]), interpolation=cv2.INTER_AREA).astype(bool)
      
      original_size = np.array(original_size)
      print(original_size.shape)
      final.append((original_size))

(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)
(586, 821)

In [55]:
predictions = []
current_state = 0
test_frames = [test_data[video]['video'].shape[2] for video in range(len(test_data))]

for i, (d,f) in enumerate(zip(test_data, test_frames)):


    prediction = np.array(np.zeros_like(d['video']), dtype=np.bool_)
    #height = prediction.shape[0]
    #width = prediction.shape[1]

    for j, p in enumerate(final[current_state:current_state+f]):
      
      prediction[:, :, j] = p
    #prediction = [(list(final[current_state:current_state+f]))
    #prediction = [p for p in final[current_state:current_state+f]]

    current_state += f

    print(len(prediction), f)
    # DATA Strucure
    predictions.append({
        'name': d['name'],
        'prediction': prediction
        }
    )

586 103
587 52
583 69
582 61
732 53
583 84
582 78
587 125
730 76
587 104
583 68
587 90
587 78
587 73
707 39
731 72
583 106
583 63
594 51
583 62


In [56]:
# save in correct format
save_zipped_pickle(predictions, 'my_predictions15.pkl')